In [ ]:
import pdb 
import coarse_functions as cfunc
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
from netCDF4 import Dataset
import time
import math
import xarray as xr
import glob
from os.path import exists
from scipy.interpolate import interp1d

import advect_scalar3D_f2py_my_min2

In [ ]:
ncfile = '/ocean/projects/ees220005p/janniy/gsam_data/SAM7.7_test/DYAMOND2_9216x4608x74_10s_4608_20200214121500_0000220410.2D_atm.nc'
f_solin = Dataset(ncfile, mode='r')
topog_high = f_solin.variables['ZSFC'][:]  # m
lat_high = f_solin.variables['lat'][:]  # m
lon_high = f_solin.variables['lon'][:]  # m
f_solin.close()

In [ ]:
number_of_files = 1
## Griffin's comment - was originally 1080
start_time=137880       
interval = 360 
end_time = start_time  + interval * (number_of_files)

In [ ]:
compare_flag = False #compare to matlab
test_mode = False #compare to matlab when writing NETCDF
test_mode1D = False #compare 1D to matlab when writing NETCDF - should be off.
res = [12]
dtn = 10

In [ ]:
experiment = 'DYAMOND2_coars_9216x4608x74_10s_4608'
loaddir = '/ocean/projects/ees220005p/janniy/gsam_data/'

if test_mode or compare_flag:
    print('To do test mode, for now I didnt write schemes')
else:
    filename = 'Dummy'

savedir = '/ocean/projects/ees220005p/gmooers/GM_Data/'
file_coarse1 = '/ocean/projects/ees220005p/janniy/gsam_data/DYAMOND2_coars_9216x4608x74_10s_4608_20200124120000_0000038880.atm.3DC.nc'
#Read hires

In [ ]:
start = time.time()
f = Dataset(file_coarse1, mode='r')
lon = f.variables['lon'][:]
lat = f.variables['lat'][:]
z = f.variables['z'][:]
zi = f.variables['zi'][:]
rho = f.variables['rho'][:]  # m
rhow2 = f.variables['rhoi'][:]  # m
p = f.variables['p'][:]  # m

f.close()

In [ ]:
rhow = np.zeros(rhow2.shape[0] + 1)
rhow[:-1] = rhow2
rhow[-1] = rhow2[-1]

#The zi in file is for some reason wrong in the sense that it is not always between the z levels). Not sure what to use
zi_from_marat = np.array ([0.0000000, 40., 82.40000, 127.3440, 174.9846,
   225.4837,       279.0128,       335.7535,       395.8987,       459.6526,
   527.2318,       598.8657,       674.7977,       755.2855,       840.6027,
   931.0388,       1026.901,       1129.591,       1242.550,       1366.805,
   1503.486,       1653.834,       1819.218,       2001.140,       2201.311,
   2431.507,       2696.233,       3000.668,       3350.769,       3753.384,
   4216.392,       4716.392,       5216.392,       5716.392,       6216.392,
   6716.392,       7216.392,       7716.392,       8216.392,       8716.392,
   9216.392,       9716.392,       10216.39,       10716.39,      11216.39,
   11716.39,       12216.39,       12716.39,       13216.39,       13716.39,
   14216.39,       14716.39,       15216.39,       15716.39,       16216.39,
   16716.39,       17216.39,       17716.39,       18216.39,       18747.30,
   19354.06,       20047.50,       20840.00,       21745.71,       22780.81,
   23963.79,       25315.76,       26815.76,       28315.76,       29815.76,
   31315.76,       32815.76,       34315.76,       35815.76,       37315.76])

zi = zi_from_marat
print('Marat printed zi that might have an unclear error - therefore, changed manually (data from Marat)')

In [ ]:
nzm = z.shape[0]
lon_size = lon.shape[0]
lat_size = lat.shape[0]
nz_size = z.shape[0]


mu = np.zeros(lat_size)
ady = np.zeros(lat_size)
muv = np.zeros(lat_size + 1)

#from setparam
ny_gl = lat.shape[0]
dy = 179.98 / ny_gl
earth_factor = 1
rad_earth = 6371229
deg2rad = np.pi / 180
dy = dy * deg2rad * rad_earth / earth_factor
for j in range(lat_size):
    mu[j] = np.cos(lat[j] * deg2rad)

mu_extend = np.zeros(lat_size + 4)
mu_extend[2:-2] = mu
mu_extend[1] = mu_extend[2] / 2  
mu_extend[0] = mu_extend[1] / 2 
mu_extend[-2] = mu_extend[-3]/2
mu_extend[-1] = mu_extend[-2]/2

In [ ]:
latv_gl_high = np.zeros(lat_high.shape[0] + 1)
latv_gl_high[0] = -90
latv_gl_high[-1] = 90

for j in range(1,latv_gl_high.shape[0]-1):
    latv_gl_high[j] = 0.5*(lat_high[j]  + lat_high[j-1])

yv_gl_glob_2_high = latv_gl_high[:]*deg2rad*rad_earth/earth_factor    

dy_high = 179.98 / lat_high.shape[0]
dy_high = dy_high * deg2rad * rad_earth / earth_factor

y_gl_glob_2_high= np.zeros(lat_high.shape[0])
for j in range(lat_high.shape[0]):
    y_gl_glob_2_high[j] = 0.5*(yv_gl_glob_2_high[j + 1]+yv_gl_glob_2_high[j])


##from setgrid
dy2_high = y_gl_glob_2_high[int(lat_high.shape[0]/2)]-y_gl_glob_2_high[int(lat_high.shape[0]/2 - 1)]
ady_glob_high3 = np.zeros(lat_high.shape[0])
for j in range( lat_high.shape[0]):
    ady_glob_high3[j] = (yv_gl_glob_2_high[j+1]-yv_gl_glob_2_high[j])/dy2_high
print('ady_glob_high3 is the correct one to use for calculating the j_start and j_end')

j_start,j_end = cfunc.calc_y_ind_edge_processor(ady_glob_high3, lat_high.shape[0], int(lat_size), ny_coarse_proc=29, coarse_fact=12, processor_points=96)

In [ ]:
# Calculating coarse topography related quantities.
path_terra_coare = '/ocean/projects/ees220005p/janniy/python_fortran_coarse_graining/f2py_global_sam/files_coarse_JY/'
terra_path = 'coarse_points.nc4'
if not os.path.isfile(path_terra_coare + terra_path):
    # Create files
    terra_points = np.zeros([lon_size, lat_size, nz_size])
    terra_tot = np.zeros([lon_size, lat_size, nz_size])
    for k in range(nz_size):
        terra_w_path = '/ocean/projects/ees220005p/janniy/gsam_data/high_res_snapshot/DYAMOND2_9216x4608x74_10s_4608_TERR_MASKS_TERRA.atm.3D.nc'
        slice_ter_tot_surf = dict()
        d = Dataset(terra_w_path, 'r')
        data = d.variables['TERRA']
        slice_ter_tot_surf['TERRA'] = data[:, k, :, :]
        del data
        d.close()

        terra_mvax = np.moveaxis(slice_ter_tot_surf['TERRA'], (0, 1, 2), (2, 1, 0))
        aa, bb = terra_num_of_points(terra_mvax, lon_size, lat_size, k, k + 1, j_start, j_end, coarseness=12)
        terra_points[:, :, k] = aa[:, :, 0]
        terra_tot[:, :, k] = bb[:, :, 0]
        print('SAVE TERRA points (TODO)')
        lat_coarse = lat
        lon_coarse = lon

    netxarr = xr.DataArray(
        data=np.moveaxis(terra_points, (0, 1, 2), (2, 1, 0)).astype(np.float32),
        dims=['z', 'lat_coarse', 'lon_coarse'],
        coords=dict(
            z=('z', z),
            lat_coarse=('lat_coarse', lat_coarse),
            lon_coarse=('lon_coarse', lon_coarse),
        ),
        attrs=dict(
            description='Number of point above terra',
            units="number",
        )
    )

    netxarr = netxarr.rename('Terra_points')
    netset = netxarr.to_dataset()

    netxarr2 = xr.DataArray(
        data=np.moveaxis(terra_tot, (0, 1, 2), (2, 1, 0)).astype(np.float32),
        dims=['z', 'lat_coarse', 'lon_coarse'],
        coords=dict(
            z=('z', z),
            lat_coarse=('lat_coarse', lat_coarse),
            lon_coarse=('lon_coarse', lon_coarse),
        ),
        attrs=dict(
            description='Total Number of point in coarse graining',
            units="number",
        )
    )
    netset['Terra_tot'] = netxarr2
    netset.to_netcdf(path=path_terra_coare + terra_path, mode='w', format='NETCDF4')